<a href="https://colab.research.google.com/github/Migaalee/Genomics_colab_HMI/blob/main/Parse_Xml_files_from_KEGG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# For general data science and matrix manipulation
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy import stats

# For pathway analysis in python
#import sspa
#import gseapy

# For plotting
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# For network creation and visualisation
import networkx as nx

#For regex
import re
import random

from google.colab import files


import sys
sys.path


import xml.etree.ElementTree as ET #Parse and read XML data
import tarfile #read from tarfile instead of extracting all data
import xml.etree.ElementTree as ET
from collections import Counter

import pprint as pp
import pickle
import site
site.getsitepackages()
from collections import defaultdict #I'm importing this library so we have a code with less lists!!

In [11]:
uploaded = files.upload()

Saving dme00190.xml to dme00190.xml


In [12]:
tree = ET.parse('dme01100.xml')
root = tree.getroot()

In [ ]:
cpd = {}
CG = {}
KO = {}


In [23]:
#for elem in root:
    #print(f"ID: {elem.attrib['id']}, Name: {elem.attrib['name']}, Type: {elem.attrib['type']}")

In [24]:
#for elem in root:
    #print(elem.tag, elem.attrib)

In [25]:
cpd_entries = []
for entry in root.findall('entry'):
    name = entry.attrib.get('name', '')  # get name attribute
    if name.startswith('cpd:'):  # if name starts with "cpd:"
        entry_details = {
            'id': entry.attrib['id'],
            'name': name,
            'type': entry.attrib['type'],
            'link': entry.attrib['link'],
        }
        cpd_entries.append(entry_details)

In [27]:
#cpd_entries

In [28]:
df = pd.DataFrame(cpd_entries)


In [29]:
df

,id,name,type,link
0,7894,cpd:C22834,compound,https://www.kegg.jp/dbget-bin/www_bget?C22834
1,7888,cpd:C03852,compound,https://www.kegg.jp/dbget-bin/www_bget?C03852
2,7887,cpd:C18041,compound,https://www.kegg.jp/dbget-bin/www_bget?C18041
3,7886,cpd:C18042,compound,https://www.kegg.jp/dbget-bin/www_bget?C18042
4,7885,cpd:C22832,compound,https://www.kegg.jp/dbget-bin/www_bget?C22832
...,...,...,...,...
3144,7889,cpd:C03917,compound,https://www.kegg.jp/dbget-bin/www_bget?C03917
3145,3643,cpd:C00606,compound,https://www.kegg.jp/dbget-bin/www_bget?C00606
3146,4072,cpd:C05527,compound,https://www.kegg.jp/dbget-bin/www_bget?C05527
3147,4323,cpd:C00053,compound,https://www.kegg.jp/dbget-bin/www_bget?C00053


No charts were generated by quickchart


In [31]:
df.to_csv("dme01100.csv", sep=";")

In [32]:
files.download("dme01100.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>